<a href="https://colab.research.google.com/github/KhudV/graduation_paper_code/blob/main/text%20generation/%D0%92%D0%9A%D0%A0_SeqGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Выборка данных:**
>*    Dataset: NetFlix Shows. Этот набор данных содержит непомеченные текстовые данные около 9000 шоу и фильмов Netflix, а также полную информацию, такую как актерский состав, год выпуска, рейтинг, описание и т. д.

**Описание работы:**
> В данной работе проведены реализация, обучение и тестирование нейронных сетей для генерации текстов. Используются такие нейросетевые модели как LSTM и SeqGAN. Проведена оценка качества генерации текста с помощью метрики BLEU.




Подключение к Google Drive

In [ ]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

path = '/content/drive/My Drive/ВКР/SeqGAN/'
os.chdir(path)

Объявление параметров по умолчанию

In [ ]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

import helpers # из Yu L. et al.

CUDA = False # использование cuda (gpu/tpu): True/False
BATCH_SIZE = 100 # количество примеров в батче
MLE_TRAIN_EPOCHS = 30 # количество эпох при обучении генератора на основе MLE
DIS_TRAIN_ITERATIONS = 300 # количество итераций при обучении дискриминатора
DIS_TRAIN_EPOCHS = 1 # количество эпох в одной итерации обучения дискриминатора
ADV_TRAIN_EPOCHS = 25 # количество эпох при обучении генератора на основе состязательного обучения / обучения с подкреплением
POS_NEG_SAMPLES = 1000

GEN_EMBEDDING_DIM = 150 # ширина слоя Embedding генератора
GEN_HIDDEN_DIM = 150 # количество нейронов в слоях генератора
DIS_EMBEDDING_DIM = 150 # ширина слоя Embedding дискриминатора
DIS_HIDDEN_DIM = 150 # количество нейронов в слоях дискриминатора

# параметры START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN и FILE_PATHS задаются ниже
START_LETTER = 0 #! стартовое слово
MAX_SEQ_LEN = 24 # длина генерируемого примера
VOCAB_SIZE = 20662 # количество слов в словаре
FILE_PATHS = {'train': r'datasets/netflix_titles_train.txt', 'test': r'datasets/netflix_titles_test.txt',
              'vocab': r'datasets/netflix_titles_vocab.pkl', 'saved_models': r'saved_models/netflix_titles'} # пути к файлам набора данных
CLOSING_WORD = 20661 # заключительное слово / слово заполнитель ставящееся в конце предложения

Инициализация генераторов случайных чисел.

In [ ]:
torch.random.manual_seed(66)
np.random.seed(66)

Объявление класса Генератора

In [ ]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # количество элементов на скрытом слое
    self.embedding_dim = embedding_dim # размер слоя embedding
    self.max_seq_len = max_seq_len # длина генерируемых примеров
    self.vocab_size = vocab_size # размер словаря использующегося при генерации
    self.gpu = gpu # использование cuda (True/False)
    self.lstm_num_layers = 1 # количество LSTM слоев

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя embeddings
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # объявление LSTM слоев
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # объявление выходного слоя

  def init_hidden(self, batch_size=1):
    # инициализация состояния LSTM слоев
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim)) 
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size

Объявяление класса Дискриминатора

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # размер скрытого слоя
        self.embedding_dim = embedding_dim # ширина слоя embedding
        self.max_seq_len = max_seq_len # длина входного примера
        self.gpu = gpu # использование cuda (True/False)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # объявление LSTM слоев

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # объявление скрытого слоя
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # объявление скрытого слоя

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # объявление выходного слоя

    def init_hidden(self, batch_size):
        # инициализация LSTM слоев
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim
        
        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Объявление функций возвращающих данные

In [ ]:
# функции генерации данных
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Функции обучения генератора

In [ ]:
# функция обучения генератора на основе MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # обучение
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # тестирование
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Тестирование генератора на обучающей и тестовой выборках.
  '''
  # тестирование на обучающей
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # тестирование на тестовой
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# функция обучения генератора на основе обучения с подкреплением RL
def train_generator_PG(gen, gen_opt, dis, num_batches):
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Функция обучения дискриминатора

In [ ]:
# функция обучения дискриминатора
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ДО ОБУЧЕНИЯ: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Функция оценки качества генерации текста

In [ ]:
# оценка качества по BLEU метрике
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# функция оценки качества генерации текста по метрике BLEU
def BLEU(reference_sample, test_sample, print_iteration=100, flag_print=True):
  if flag_print:
    print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  mass_bleu = []
  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0 and flag_print:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))
          
      mass_bleu.append(1.0 * sum(bleu_leakgan) / len(bleu_leakgan))
      if flag_print:
        print('--- --- ---')
        print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")
  return mass_bleu

Функция сохранения моделей

In [ ]:
# функция сохранения моделей (сериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Функция загрузки моделей

In [ ]:
# функция загрузки моделей (десериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
  
  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Загрузка набора данных

In [ ]:
# загрузка словаря
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [ ]:
# загрузка обучающей выборки
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Примеров в обучающей выборке: ", len(data_file_train))

Примеров в обучающей выборке:  8704


In [ ]:
# загрузка тестовой выборки
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Примеров в тестовой выборке: ", len(data_file_test))

Примеров в тестовой выборке:  968


In [ ]:
# примеры из обучающей выборки
print("Примеры из обучающей выборки")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры из обучающей выборки
# 0 	Пример:  after learning of their son 's death on the battlefield , a grieving berlin couple embark on a quietly dangerous act of resistance against
# 1 	Пример:  the pythons elevate the absurd to new heights and bring their sketches to german tv , working in phonetic german , at times with
# 2 	Пример:  but after maddy is betrayed , she and cole reunite – to ruin prom .         
# 3 	Пример:  when law student darby shaw theorizes about the assassinations of two supreme court justices , she is put in danger and turns to a
# 4 	Пример:  fighting to survive , he reflects on his life and past romance .           
# 5 	Пример:  two childhood best friends reunite as an unlikely crime-fighting superhero duo when one invents a formula that gives ordinary people superpowers .  
# 6 	Пример:  a penniless country boy goes in search of his runaway sister in bogotá , where he falls for an aspiring singer , but gets
# 7 	Пример:  kat and eva ’ s happy romance hits a

Создание нейронных сетей генератора и дискриминатора

In [ ]:
# объявление нейронных сетей генератора и дискриминатора, подготовка выборок данных для использования pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) #, lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Обучение генератора на основе MLE

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 9.9228 average_test_NLL = 9.9233


In [ ]:
# обучение генератора на основе MLE / предобучение генератора
print('Запуск обучения генератора на основе MLE...')
gen_optimizer = optim.Adam(gen.parameters())#, lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, MLE_TRAIN_EPOCHS) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)

Запуск обучения генератора на основе MLE...
epoch 1 : .......... average_train_NLL = 7.3663 average_test_NLL = 6.5351
epoch 2 : .......... average_train_NLL = 6.3086 average_test_NLL = 6.3586
epoch 3 : .......... average_train_NLL = 6.0647 average_test_NLL = 6.1910
epoch 4 : .......... average_train_NLL = 5.8529 average_test_NLL = 6.0663
epoch 5 : .......... average_train_NLL = 5.6757 average_test_NLL = 5.9727
epoch 6 : .......... average_train_NLL = 5.5215 average_test_NLL = 5.8997
epoch 7 : .......... average_train_NLL = 5.3839 average_test_NLL = 5.8430
epoch 8 : .......... average_train_NLL = 5.2591 average_test_NLL = 5.7996
epoch 9 : .......... average_train_NLL = 5.1441 average_test_NLL = 5.7657
epoch 10 : .......... average_train_NLL = 5.0365 average_test_NLL = 5.7399
epoch 11 : .......... average_train_NLL = 4.9347 average_test_NLL = 5.7202
epoch 12 : .......... average_train_NLL = 4.8374 average_test_NLL = 5.7046
epoch 13 : .......... average_train_NLL = 4.7436 average_test_NLL

RuntimeError: ignored

In [ ]:
# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 3.4587 average_test_NLL = 5.7914


Генерация примеров текстов на основе MLE

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1
# 0 	Пример:  a district cow as a newly account to the first business terrain , an natural adventure uses the time can their secret kimchee differences  	Оценка:  [0.48900964692182575, 0.10281837227019264, 0.047697785860210014, 0.03038033625841874]
# 1 	Пример:  the street final performance at stake at his life choices .                                           	Оценка:  [0.8846517369293828, 0.7926534264491066, 0.7451496818790302, 0.7135510776575065]
# 2 	Пример:  this documentary explores the true life and producers of skull castle in this entertaining supply italy series .        	Оценка:  [0.6689273264831853, 0.5459359922380247, 0.4643434078111817, 0.39235340559600385]
# 3 	Пример:  this hindi outback album of musicians characters to the classic moments of the rest of teachers with his mother 's body , a thankless  	Оценка:  [0.6313087395543334, 0.3787861220823229, 0.1268354694168578, 0.06643341491719774]
# 4 	Пример:  an actor i

Оценка качества генерации текста после обучения с помощью MLE

In [ ]:
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.5948866545792448
2 200 0.6054552359457742
2 300 0.6016202455435079
2 400 0.6005756808019966
2 500 0.604328182451673
--- --- ---
2 -gram BLEU score :  0.604328182451673 

3 100 0.33878084409882786
3 200 0.35751802465740956
3 300 0.3604365532516006
3 400 0.3588395654152625
3 500 0.3626909506096581
--- --- ---
3 -gram BLEU score :  0.3626909506096581 

4 100 0.23614141588392815
4 200 0.2538859337269141
4 300 0.25550202397863003
4 400 0.25226020522824966
4 500 0.2537040848961532
--- --- ---
4 -gram BLEU score :  0.2537040848961532 

5 100 0.19438796915345255
5 200 0.20851230711370494
5 300 0.21056530155429862
5 400 0.2073277189558224
5 500 0.20767160383787964
--- --- ---
5 -gram BLEU score :  0.20767160383787964 



[0.604328182451673,
 0.3626909506096581,
 0.2537040848961532,
 0.20767160383787964]

Предварительное обучение дискриминатора

In [ ]:
# предобучение дискриминатора
print('Запуск обучения дискриминатора...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # регуляризация
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)#25, 1 | (15, 3), (25, 1)

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ДО ОБУЧЕНИЯ: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Запуск обучения дискриминатора...
ДО ОБУЧЕНИЯ: val_acc = 0.4980
d-step 1 epoch 1 : .......... average_loss = 0.3812, train_acc = 0.8870, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 2 epoch 1 : .......... average_loss = 0.3481, train_acc = 0.8900, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 3 epoch 1 : .......... average_loss = 0.3411, train_acc = 0.8905, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 4 epoch 1 : .......... average_loss = 0.3116, train_acc = 0.9020, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 5 epoch 1 : .......... average_loss = 0.3053, train_acc = 0.9020, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 6 epoch 1 : .......... average_loss = 0.3233, train_acc = 0.8970, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 7 epoch 1 : .......... average_loss = 0.3377, train_acc = 0.8885, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 8 epoch 1 : .......... average_loss = 0.3399, train_acc = 0.8885, val_acc = 0.5000
ДО ОБУ

Состязательное обучение. Обучение генератора на основе обучения с подкреплением

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 3.4587 average_test_NLL = 5.7914


In [ ]:
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# состязательное обучение генератора
print('\nStarting Adversarial Training...')

for epoch in range(ADV_TRAIN_EPOCHS):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # обучение генератора
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)#

    # тестирование nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # обучение дискриминатора
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 3.4626 average_test_NLL = 5.7944

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val_acc = 0.9210
d-step 1 epoch 1 : .......... average_loss = 0.0776, train_acc = 0.9765, val_acc = 0.9210
ДО ОБУЧЕНИЯ: val_acc = 0.9150
d-step 2 epoch 1 : .......... average_loss = 0.0508, train_acc = 0.9885, val_acc = 0.9190
ДО ОБУЧЕНИЯ: val_acc = 0.9200
d-step 3 epoch 1 : .......... average_loss = 0.0595, train_acc = 0.9850, val_acc = 0.9250
ДО ОБУЧЕНИЯ: val_acc = 0.9260
d-step 4 epoch 1 : .......... average_loss = 0.0626, train_acc = 0.9850, val_acc = 0.9220
ДО ОБУЧЕНИЯ: val_acc = 0.9230
d-step 5 epoch 1 : .......... average_loss = 0.0594, train_acc = 0.9870, val_acc = 0.9360

--------
EPOCH 2
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 3.4677 average_test_NLL = 5.7976

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val

Тестирование генератора на обучающей выборке после состязательного обучения.

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 3.5454 average_test_NLL = 5.8308


Генерация примеров текстов на основе SeqGAN

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе SeqGAN
Degree: 1
# 0 	Пример:  in jesse writes face off with 25 opportunity in a remote village while dealing with an heiress and karaoke when she must his help  	Оценка:  [0.6313087395543334, 0.4169080726689162, 0.2423680701003884, 0.11152553740468045]
# 1 	Пример:  a new professor reunites in spanish business murders , the director and death of gender sides .         	Оценка:  [0.6915640748081247, 0.5338837915749063, 0.4362874590298468, 0.37327094197706856]
# 2 	Пример:  when his eyes takes his death , she 's forced to return to her , two sons on the autism kingdom to his true  	Оценка:  [0.7801894976054939, 0.30245388888833086, 0.10713701843513149, 0.05804243793890311]
# 3 	Пример:  a engaged village slave artist who shows him to korea and demonic protection in the gang to beijing .       	Оценка:  [0.5646597025732799, 0.38702317094861327, 0.30166728533047465, 0.24189259416605804]
# 4 	Пример:  a quiet love comedian raised by an eccentric frien

Оценка качества генерации на основе SeqGAN

In [ ]:
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 1
--- --- ---
Start BLEU
2 100 0.6092547022852403
2 200 0.6205699445641378
2 300 0.6207494020121411
2 400 0.623373852981553
2 500 0.6303386721629859
--- --- ---
2 -gram BLEU score :  0.6303386721629859 

3 100 0.36357426141964105
3 200 0.3792089713898284
3 300 0.38303278234081245
3 400 0.3847275959367494
3 500 0.3938521511328785
--- --- ---
3 -gram BLEU score :  0.3938521511328785 

4 100 0.25206721888454864
4 200 0.26947934729175205
4 300 0.27363160716197876
4 400 0.275768289431875
4 500 0.2834170040187484
--- --- ---
4 -gram BLEU score :  0.2834170040187484 

5 100 0.20007553907992942
5 200 0.2190419075984661
5 300 0.222988038605337
5 400 0.2261766857827814
5 500 0.23296601500720862
--- --- ---
5 -gram BLEU score :  0.23296601500720862 



[0.6303386721629859,
 0.3938521511328785,
 0.2834170040187484,
 0.23296601500720862]

LSTM: [0.604328182451673,
 0.3626909506096581,
 0.2537040848961532,
 0.20767160383787964]

 SeqGAN: [0.6303386721629859,
 0.3938521511328785,
 0.2834170040187484,
 0.23296601500720862]

Загрузка сохраненной модели

In [ ]:
# загрузка моделей
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()